# Cost Function and Backpropagation

Now, we're going to study an algorithm for fitting the parameters of a Neural Network, known as the Backpropagation algorithm. Before doing that, we will define the cost function

## 1. Cost Function

To define the cost function for Neural Networks, let's consider the following general neural network with $L$ layers for **classification problems**:

![NN](figures/neural_network_general.png)

Moreover, we have $m$ training examples $\{(x^{(1)}, y^{(1)}), \dots (x^{(m)}, y^{(m)})\}$.

- For binary classification we would have $y\in\{0,1\}$, and $K=1$ output units.

- For multi-class classification we have $y\in\mathbb{R}^K$ and $K$ output units. For example, for $K=4$:
  $$
  y = \left[\begin{array}{c}
  1 \\
  0 \\
  0 \\
  0
  \end{array}\right], \left[\begin{array}{c}
  0 \\
  0 \\
  1 \\
  0
  \end{array}\right]
  $$

### What happens from layer $l$ to layer $l+1$ ($l\in\{1,\dots,L-1\}$)?

![transition](figures/transition.png)

Recall that we introduce the intermediate variables $z_j^{(l+1)}$ for $j\in\{1, \dots, s_{l+1}\}$, defined as

$$
z_j^{(l+1)} = \Theta_{j0}^{(l)} + \Theta_{j0}^{(l)}a_1^{(l)} + \dots + \Theta_{js_l}^{(l)} a_{s_l}^{(l)}.
$$

Then, the $j$-th unit activation is defined as

$$
a_j^{(l+1)} = g(z_j^{(l+1)}),
$$

where $g$ is the sigmoid function.

Equivalently,

\begin{align}
z^{(l+1)} & = \Theta^{(l)} \left[\begin{array}{c} 1 \\ a^{(l)} \end{array}\right] \\
a^{(l+1)} & = g(z^{(l+1)}),
\end{align}

where

$$
\Theta^{(l)} = \left[
\begin{array}{cccc}
\Theta_{10}^{(l)}       & \Theta_{11}^{(l)}       & \dots  & \Theta_{1s_l}^{(l)}  \\
\Theta_{20}^{(l)}       & \Theta_{21}^{(l)}       & \dots  & \Theta_{2s_l}^{(l)}  \\
\vdots                  & \vdots                  & \ddots & \vdots               \\
\Theta_{s_{l+1}0}^{(l)} & \Theta_{s_{l+1}1}^{(l)} & \dots  & \Theta_{s_{l+1}s_l}^{(l)} 
\end{array}\right],
$$

### What is the form of the hypothesis function?

Following the above analysis, in the output layer $L$:

$$\left\{
\begin{align}
z^{(L)} & = \Theta^{(L-1)} \left[\begin{array}{c} 1 \\ a^{(L-1)} \end{array}\right] \in\mathbb{R}^{K} \\
a^{(L)} & = h_{\Theta}(x) = g(z^{(L)})\in\mathbb{R}^{K}.
\end{align}\right.
$$

In the hidden layers $l = 2,\dots, L-1$:

$$\left\{
\begin{align}
z^{(l)} & = \Theta^{(l-1)} \left[\begin{array}{c} 1 \\ a^{(l-1)} \end{array}\right] \in\mathbb{R}^{s_l} \\
a^{(l)} & = g(z^{(l})\in\mathbb{R}^{s_l}.
\end{align}\right.
$$

In the input layer:

$$
a^{(1)} = x \in\mathbb{R}^{n}.
$$

### What is the form of the cost function?

The cost function we use for this class of neural networks is a generalization of the cost function for the logistic regression:

$$
J(\Theta) = -\frac{1}{m} \left[\sum_{t=1}^{m} \sum_{k=1}^{K} y^{(t)}_k \log(h_{\Theta}(x^{(t)})_k) + (1-y^{(t)}_k) \log(1 - h_{\Theta}(x^{(t)})_k)\right] + \frac{\lambda}{2m} \sum_{l=1}^{L-1} \sum_{i=1}^{s_{l+1}} \sum_{j=1}^{s_l} \left(\Theta_{ij}^{l}\right)^2
$$

Thus, in order to use a gradient based optimization method, we need to compute

$$
\frac{\partial}{\partial \Theta_{ij}^{(l)}} J(\Theta),
$$

for each $i\in\{1, \dots, s_{l+1}\}$, $j\in\{0, \dots, s_{l}\}$, and $l\in\{1, \dots, L-1\}$.

Defining the *cost for the example $t$ at the output $k$*, $c_k^{(t)}$ as:

$$
c_k^{(t)} = -(y^{(t)}_k \log(h_{\Theta}(x^{(t)})_k) + (1-y^{(t)}_k) \log(1 - h_{\Theta}(x^{(t)})_k),
$$

we have that

$$
J(\Theta) = \frac{1}{m} \left[\sum_{t=1}^{m} \sum_{k=1}^{K} c_k^{(t)}\right] + \frac{\lambda}{2m} \sum_{l=1}^{L-1} \sum_{i=1}^{s_{l+1}} \sum_{j=1}^{s_l} \left(\Theta_{ij}^{(l)}\right)^2.
$$

## 2. Computing the Gradient

Before starting our analysis, let's note that for the sigmoid function

$$g(z) = \frac{1}{1+e^{-z}},$$

its derivative is

$$
g'(z) = \frac{e^{-z}}{(1+e^{-z})^2} = \frac{1+e^{-z} - 1}{(1+e^{-z})^2} = g(z) - g(z)^2 = g(z) (1 - g(z)).
$$

### First step: For $l=L-1$

Let $i\in\{1, \dots, s_L=K\}$ and $j\in\{0, \dots, s_{L-1}\}$. Then,

$$
\frac{\partial}{\partial \Theta_{ij}^{(L-1)}} J(\Theta) = \frac{1}{m} \left[\sum_{t=1}^{m} \sum_{k=1}^{K} \frac{\partial}{\partial \Theta_{ij}^{(L-1)}}c_k^{(t)}\right] + \frac{\lambda}{m} \Theta_{ij}^{(L-1)} \mathcal{I}\{j>0\},
$$

where $\mathcal{I}(\cdot)$ stands for the indicator function.

![L-1](figures/L-1.png)

Clearly, 

$$
\frac{\partial}{\partial \Theta_{ij}^{(L-1)}}c_k^{(t)} = 0,
$$

whenever $k \neq i$. On the other hand, using the chain rule

$$
\frac{\partial}{\partial \Theta_{ij}^{(L-1)}}c_i^{(t)} = \frac{\partial}{\partial z_{i}^{(t,L)}}c_i^{(t)} \frac{\partial}{\partial \Theta_{ij}^{(L-1)}} z_{i}^{(t,L)},
$$

where ($h_{\Theta}(x^{(t)})_k=a_k^{(t,L)}=g(z_k^{(t,L)})$)

\begin{align}
\frac{\partial}{\partial z_{i}^{(t,L)}}c_i^{(t)} & = y^{(t)}_i \frac{1}{a^{(t,L)}_i} a^{(t,L)}_i (1 - a^{(t,L)}_i) + (1-y^{(t)}_i) \frac{1}{1 - a^{(t,L)}_i} (-a^{(t,L)}_i (1 - a^{(t,L)}_i)) \\
& = y^{(t)}_i (1 - a^{(t,L)}_i) - (1-y^{(t)}_i) a^{(t,L)}_i \\
& = a^{(t,L)}_i - y_i^{(t)} \\
& =: \delta_i^{(t,L)},
\end{align}

and $\frac{\partial}{\partial \Theta_{ij}^{(L-1)}} z_{i}^{(t,L)} = a_j^{(t,L-1)}$.

Hence,

$$
\frac{\partial}{\partial \Theta_{ij}^{(L-1)}} J(\Theta) = \frac{1}{m} \left[\sum_{t=1}^{m} \delta_i^{(t,L)} a_j^{(t,L-1)}\right] + \frac{\lambda}{m} \Theta_{ij}^{(L-1)} \mathcal{I}\{j>0\}.
$$

### Second step: For $l=L-2$

Let $i\in\{1, \dots, s_{L-1}\}$ and $j\in\{0, \dots, s_{L-2}\}$. Then,

Then,

$$
\frac{\partial}{\partial \Theta_{ij}^{(L-2)}} J(\Theta) = \frac{1}{m} \sum_{t=1}^{m} \frac{\partial}{\partial \Theta_{ij}^{(L-2)}}\left[\sum_{k=1}^{K} c_k^{(t)}\right] + \frac{\lambda}{m} \Theta_{ij}^{(L-2)} \mathcal{I}\{j>0\}.
$$

It is worth to notice here that every unit in the output layer $L$ depends on $\Theta_{ij}^{(L-2)}$, for every $i, j, k$:

![L-2](figures/L-2.png)

Hence, applying the chain rule

$$
 \frac{\partial}{\partial \Theta_{ij}^{(L-2)}}\left[\sum_{k=1}^{K} c_k^{(t)}\right] = \frac{\partial}{\partial z_{i}^{(t,L-1)}}\left[\sum_{k=1}^{K} c_k^{(t)}\right] \frac{\partial}{\partial \Theta_{ij}^{(L-2)}} z_{i}^{(t,L-1)}
$$

where

\begin{align}
\frac{\partial}{\partial z_{i}^{(t,L-1)}}\left[\sum_{k=1}^{K} c_k^{(t)}\right] & = \sum_{k=1}^{K}\frac{\partial}{\partial z_{k}^{(t,L)}}c_k^{(t)} \frac{\partial}{\partial a_{i}^{(t,L-1)}}z_k^{(t)} \frac{\partial}{\partial z_{i}^{(t,L-1)}}a_{i}^{(t,L-1)} \\
& = \left[\sum_{k=1}^{K} \delta_k^{(t, L)} \Theta_{ki}^{(L-1)}\right] a_i^{(t,L-1)} (1 - a_i^{(t,L-1)})  \\
& = (\Theta_{:i}^{(L-1)})^T \delta^{(t, L)} a_i^{(t,L-1)} (1 - a_i^{(t,L-1)}) \\
& =: \delta_i^{(t, L-1)},
\end{align}

where $(\Theta_{:i}^{(L-1)})^T = \left[\Theta_{1i}^{(L-1)} \quad \Theta_{2i}^{(L-1)} \quad \dots \quad \Theta_{s_{L}i}^{(L-1)}\right]$ is the $i$-th column of $\Theta^{(L-1)}$ transposed, and $\delta^{(t, L)} = \left[\delta_1^{(t, L)} \quad \delta_2^{(t, L)} \quad \dots \quad \delta_{K}^{(t, L)}\right]$.

Moreover, $\frac{\partial}{\partial \Theta_{ij}^{(L-2)}} z_{i}^{(t,L-1)} = a_j^{(t, L-2)}$.

Thus,

$$
\frac{\partial}{\partial \Theta_{ij}^{(L-2)}} J(\Theta) = \frac{1}{m} \left[\sum_{t=1}^{m} \delta_i^{(t, L-1)} a_j^{(t, L-2)}\right] + \frac{\lambda}{m} \Theta_{ij}^{(L-2)} \mathcal{I}\{j>0\}.
$$

### Inductive step

Suppose that we've calculated all the partial derivatives of the cost function w.r.t. the parameters in the layer $l+1$ as

$$
\frac{\partial}{\partial \Theta_{ij}^{(l+1)}} J(\Theta) = \frac{1}{m} \left[\sum_{t=1}^{m} \delta_i^{(t, l+2)} a_j^{(t, l+1)}\right] + \frac{\lambda}{m} \Theta_{ij}^{(l+1)} \mathcal{I}\{j>0\}.
$$

for all $i\in\{1, \dots, s_{l+2}\}$ and $j\in\{0, \dots, s_{l+1}\}$, with

\begin{align}
\delta_i^{(t, l+2)} &:= \frac{\partial}{\partial z_{i}^{(t,L-1)}} \left[\sum_{k=1}^{K} c_k^{(t)}\right] \\
& = (\Theta_{:i}^{(l+2)})^T \delta^{(t, l+3)} a_i^{(t,l+2)} (1 - a_i^{(t,l+2)}).
\end{align}

Now, for the layer $l$, let $i\in\{1, \dots, s_{l+1}\}$ and $j\in\{0, \dots, s_{l}\}$. Then,

$$
\frac{\partial}{\partial \Theta_{ij}^{(l)}} J(\Theta) = \frac{1}{m} \sum_{t=1}^{m} \frac{\partial}{\partial \Theta_{ij}^{(l)}}\left[\sum_{k=1}^{K} c_k^{(t)}\right] + \frac{\lambda}{m} \Theta_{ij}^{(l)} \mathcal{I}\{j>0\}.
$$

![l](figures/l.png)

Applying the chain rule, we have

$$
\frac{\partial}{\partial \Theta_{ij}^{(l)}}\left[\sum_{k=1}^{K} c_k^{(t)}\right] = \frac{\partial}{\partial z_{i}^{(t,l+1)}}\left[\sum_{k=1}^{K} c_k^{(t)}\right] \frac{\partial}{\partial \Theta_{ij}^{(l)}} z_{i}^{(t,l+1)},
$$

where

\begin{align}
\frac{\partial}{\partial z_{i}^{(t,l+1)}}\left[\sum_{k=1}^{K} c_k^{(t)}\right] &=\sum_{p=1}^{s_{l+2}} \left( \frac{\partial}{\partial z_{p}^{(t,l+2)}}\left[\sum_{k=1}^{K} c_k^{(t)}\right] \frac{\partial}{\partial a_{i}^{(t,l+1)}} z_{p}^{(t,l+2)} \right) \frac{\partial}{\partial z_{i}^{(t,l+1)}} a_{i}^{(t,l+1)}  \\
& = \left[\sum_{p=1}^{s_{l+2}} \delta_p^{(t, l+2)} \Theta_{pi}^{(l+1)}\right] a_i^{(t,l+1)} (1 - a_i^{(t,l+1)}) \\
& = (\Theta_{:i}^{(l+1)})^T \delta^{(t, l+2)} a_i^{(t,l+1)} (1 - a_i^{(t,l+1)}) \\
& =: \delta_i^{(t,l+1)},
\end{align}

and $\frac{\partial}{\partial \Theta_{ij}^{(l)}} z_{i}^{(t,l+1)} = a_j^{(t,l)}$.

Thus, 

$$
\frac{\partial}{\partial \Theta_{ij}^{(l)}} J(\Theta) = \frac{1}{m} \left[\sum_{t=1}^{m} \delta_i^{(t, l+1)} a_j^{(t, l)}\right] + \frac{\lambda}{m} \Theta_{ij}^{(l)} \mathcal{I}\{j>0\}.
$$

## 3. Backpropagation algorithm

Based on the analysis of the gradients above, one can come up with the following backpropagation algorithm:

Given the training set $\{(x^{(1)}, y^{(1)}), \dots (x^{(m)}, y^{(m)})\}$:

> - Set $D^{(l)} = 0\in\mathbb{R}^{s_{l+1} \times (1 + s_l)}$ for $l\in\{1, \dots, L-1\}$.
> - For $t=1, \dots, m$:
>     - Set $a^{(1)} = x^{(t)}$.
>     - Forward propagation algorithm:
>       $$\left\{
        \begin{align}
        z^{(l)} & = \Theta^{(l-1)} \left[\begin{array}{c} 1 \\ a^{(l-1)} \end{array}\right] \in\mathbb{R}^{s_l} \\
        a^{(l)} & = g(z^{(l})\in\mathbb{R}^{s_l}.
        \end{align}\right.
        $$
>       for $l\in\{2, \dots, L\}$.
>     - Compute $\delta^{(L)} = a^{(L)} - y^{(t)} \in \mathbb{R}^{s_L}$.
>     - Compute $\delta^{(l)} = (\Theta_{:,1:}^{(l)})^T \delta^{(l+1)} \circ a^{(l)} \circ (1 - a^{(l)})$, for $l\in\{L-1, \dots, 2\}$.
>     - Update $D^{(l)} := D^{(l)} + \delta^{(l+1)} \left[1 \quad (a^{(l)})^T\right]$, for $l\in\{1, \dots, L-1\}$.     
> - Update $D^{(l)} := \frac{1}{m} \left(D^{(l)} + \lambda \left[0_{s_{l+1} \times 1} \quad | \quad 1_{s_{l+1} \times s_l}\right] \circ \theta^{(l)}\right)$, for $l\in\{1, \dots, L-1\}$.

Finally, we get that

$$
\frac{\partial}{\partial \Theta_{ij}^{(l)}} J(\Theta) = D^{(l)}_{ij}
$$

<script>
  $(document).ready(function(){
    $('div.prompt').hide();
    $('div.back-to-top').hide();
    $('nav#menubar').hide();
    $('.breadcrumb').hide();
    $('.hidden-print').hide();
  });
</script>

<footer id="attribution" style="float:right; color:#808080; background:#fff;">
Created with Jupyter by Esteban Jiménez Rodríguez. Based on the content of the Machine Learning course offered through coursera by Prof. Andrew Ng.
</footer>